#  CIFAR-10 Autoencoder

Create a convolutional autoencoder for the [CIFAR10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset. 

## Imports

In [ ]:
# Install packages for compatibility with the autograder
!pip install tensorflow==2.6.0 --quiet
!pip install keras==2.6.0 --quiet

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

## Load and prepare the dataset

The [CIFAR 10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset already has train and test splits and you can use those in this exercise. 

In [ ]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024




# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load('cifar10', split='train', as_supervised=True)

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load('cifar10', split='test', as_supervised=True)

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)

# batch the dataset
test_dataset = test_dataset.batch(BATCH_SIZE)


## Build the Model



In [ ]:
# suggested layers to use. feel free to add or remove as you see fit.
from keras.layers import Conv2D, UpSampling2D

# use the Sequential API (you can remove if you want to use the Functional API)
model = Sequential()


# use `model.add()` to add layers (if using the Sequential API)
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="relu", input_shape=(32, 32, 3))) # (16, 16, 64)
model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="relu")) # (8, 8, 128)

model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")) # (8, 8, 256)

model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")) # (16, 16, 128)
model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu")) # (32, 32, 64)
model.add(UpSampling2D(size=(2, 2)))

model.add(Conv2D(filters=3, kernel_size=(3, 3), padding="same", activation="sigmoid")) # (32, 32, 1)



model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')



model.summary()

## Configure training parameters



In [ ]:

model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

## Training



In [ ]:
# parameters 
train_steps = len(train_dataset) // BATCH_SIZE 
val_steps = len(test_dataset) // BATCH_SIZE


model.fit(train_dataset, 
          steps_per_epoch=train_steps, 
          validation_data=test_dataset,
          validation_steps=val_steps, 
          epochs=50)


## Model evaluation



In [ ]:
result = model.evaluate(test_dataset, steps=10)

<img src="https://drive.google.com/uc?export=view&id=12Fy-guiP-3tTPfc9IV2nHhqLvs7LwRo6" width="75%" height="75%"/>

## Save the model



In [ ]:
model.save('mymodel.h5')